# TP 1: LDA/QDA y optimización matemática de modelos

# Intro teórica

## Definición: Clasificador Bayesiano

Sean $k$ poblaciones, $x \in \mathbb{R}^p$ puede pertenecer a cualquiera $g \in \mathcal{G}$ de ellas. Bajo un esquema bayesiano, se define entonces $\pi_j \doteq P(G = j)$ la probabilidad *a priori* de que $X$ pertenezca a la clase *j*, y se **asume conocida** la distribución condicional de cada observable dado su clase $f_j \doteq f_{X|G=j}$.

De esta manera dicha probabilidad *a posteriori* resulta
$$
P(G|_{X=x} = j) = \frac{f_{X|G=j}(x) \cdot p_G(j)}{f_X(x)} \propto f_j(x) \cdot \pi_j
$$

La regla de decisión de Bayes es entonces
$$
H(x) \doteq \arg \max_{g \in \mathcal{G}} \{ P(G|_{X=x} = j) \} = \arg \max_{g \in \mathcal{G}} \{ f_j(x) \cdot \pi_j \}
$$

es decir, se predice a $x$ como perteneciente a la población $j$ cuya probabilidad a posteriori es máxima.

*Ojo, a no desesperar! $\pi_j$ no es otra cosa que una constante prefijada, y $f_j$ es, en su esencia, un campo escalar de $x$ a simplemente evaluar.*

## Distribución condicional

Para los clasificadores de discriminante cuadrático y lineal (QDA/LDA) se asume que $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma_j)$, es decir, se asume que cada población sigue una distribución normal.

Por definición, se tiene entonces que para una clase $j$:
$$
f_j(x) = \frac{1}{(2 \pi)^\frac{p}{2} \cdot |\Sigma_j|^\frac{1}{2}} e^{- \frac{1}{2}(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)}
$$

Aplicando logaritmo (que al ser una función estrictamente creciente no afecta el cálculo de máximos/mínimos), queda algo mucho más práctico de trabajar:

$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$

Observar que en este caso $C=-\frac{p}{2} \log(2\pi)$, pero no se tiene en cuenta ya que al tener una constante aditiva en todas las clases, no afecta al cálculo del máximo.

## LDA

En el caso de LDA se hace una suposición extra, que es $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma)$, es decir que las poblaciones no sólo siguen una distribución normal sino que son de igual matriz de covarianzas. Reemplazando arriba se obtiene entonces:

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva y, distribuyendo y reagrupando términos sobre $(x-\mu_j)^T \Sigma^{-1} (x- \mu_j)$ se obtiene finalmente:

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$

## Entrenamiento/Ajuste

Obsérvese que para ambos modelos, ajustarlos a los datos implica estimar los parámetros $(\mu_j, \Sigma_j) \; \forall j = 1, \dots, k$ en el caso de QDA, y $(\mu_j, \Sigma)$ para LDA.

Estos parámetros se estiman por máxima verosimilitud, de manera que los estimadores resultan:

* $\hat{\mu}_j = \bar{x}_j$ el promedio de los $x$ de la clase *j*
* $\hat{\Sigma}_j = s^2_j$ la matriz de covarianzas estimada para cada clase *j*
* $\hat{\pi}_j = f_{R_j} = \frac{n_j}{n}$ la frecuencia relativa de la clase *j* en la muestra
* $\hat{\Sigma} = \frac{1}{n} \sum_{j=1}^k n_j \cdot s^2_j$ el promedio ponderado (por frecs. relativas) de las matrices de covarianzas de todas las clases. *Observar que se utiliza el estimador de MV y no el insesgado*

Es importante notar que si bien todos los $\mu, \Sigma$ deben ser estimados, la distribución *a priori* puede no inferirse de los datos sino asumirse previamente, utilizándose como entrada del modelo.

## Predicción

Para estos modelos, al igual que para cualquier clasificador Bayesiano del tipo antes visto, la estimación de la clase es por método *plug-in* sobre la regla de decisión $H(x)$, es decir devolver la clase que maximiza $\hat{f}_j(x) \cdot \hat{\pi}_j$, o lo que es lo mismo $\log\hat{f}_j(x) + \log\hat{\pi}_j$.

# Código provisto

Con el fin de no retrasar al alumno con cuestiones estructurales y/o secundarias al tema que se pretende tratar, se provee una base de código que **no es obligatoria de usar** pero se asume que resulta resulta beneficiosa.

In [137]:
import numpy as np
import pandas as pd
import numpy.linalg as LA
from scipy.linalg import cholesky, solve_triangular
from scipy.linalg.lapack import dtrtri

## Base code

In [138]:
class BaseBayesianClassifier:
  def __init__(self):
    pass

  def _estimate_a_priori(self, y):
    a_priori = np.bincount(y.flatten().astype(int)) / y.size
    #print("a_priori shape= ", a_priori.shape)
    #print("a_priori= ", a_priori)
    # Q3: para que sirve bincount?
    # R3: Crea un vector con la cantidad de ocurrencias de cada entero dentro de y
    return np.log(a_priori)

  def _fit_params(self, X, y):
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)

    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    # Q4: por que el _fit_params va al final? no se puede mover a, por ejemplo, antes de la priori?
    # R4: porque el _fit_params depende de la priori, y la priori se calcula antes

  def predict(self, X):
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1]
    y_hat = np.empty(m_obs, dtype=int)
    #print("y_hat shape= ", y_hat.shape)
    #print("y_hat= ", y_hat)

    for i in range(m_obs):
      y_hat[i] = self._predict_one(X[:,i].reshape(-1,1))

    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i
                  in enumerate(self.log_a_priori) ]
    # return the class that has maximum a posteriori probability
    #print("log_posteriori= ", log_posteriori)
    return np.argmax(log_posteriori)

In [139]:
class QDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate each covariance matrix
    print("X shape= ", X.shape)
    print("y shape= ", y.shape)
    # print("y.flatten", y.flatten())
    self.inv_covs = [LA.inv(np.cov(X[:,y.flatten()==idx], bias=True))
                      for idx in range(len(self.log_a_priori))]
    # Q5: por que hace falta el flatten y no se puede directamente X[:,y==idx]?
    # R5: porque y es un vector columna y no se puede comparar con un vector fila. 
    # Flatten en este caso lo convierte en arreglo de 1D y entonces si se puede hacer la comparacion con idx.
    # Q6: por que se usa bias=True en vez del default bias=False?
    # R6: porque bias=False hace que la matriz de covarianza sea insesgada.

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]
    # Q7: que hace axis=1? por que no axis=0?
    # R7: axis=1 indica que el promedio se debe calcular por columnas y no por filas.

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(LA.det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

In [140]:
class TensorizedQDA(QDA):

    def _fit_params(self, X, y):
        # ask plain QDA to fit params
        super()._fit_params(X,y)

        # stack onto new dimension
        self.tensor_inv_cov = np.stack(self.inv_covs)
        self.tensor_means = np.stack(self.means)

    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.tensor_means
        inner_prod = unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_x

        return 0.5*np.log(LA.det(self.tensor_inv_cov)) - 0.5 * inner_prod.flatten()

    def _predict_one(self, x):
        # return the class that has maximum a posteriori probability
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))

In [141]:
class QDA_Chol1(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.L_invs = [
        inv(cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True))
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L_inv = self.L_invs[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = L_inv @ unbiased_x

    return np.log(L_inv.diagonal().prod()) -0.5 * (y**2).sum()

In [142]:
class QDA_Chol2(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.Ls = [
        cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True)
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L = self.Ls[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = solve_triangular(L, unbiased_x, lower=True)

    return -np.log(L.diagonal().prod()) -0.5 * (y**2).sum()

In [143]:
class QDA_Chol3(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.L_invs = [
        dtrtri(cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True), lower=1)[0]
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L_inv = self.L_invs[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = L_inv @ unbiased_x

    return np.log(L_inv.diagonal().prod()) -0.5 * (y**2).sum()

## Datasets

In [144]:
from sklearn.datasets import load_iris, fetch_openml, load_wine
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def get_iris_dataset():
  data = load_iris()
  X_full = data.data
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
  return X_full, y_full

def get_penguins_dataset():
    # get data
    df, tgt = fetch_openml(name="penguins", return_X_y=True, as_frame=True, parser='auto')

    # drop non-numeric columns
    df.drop(columns=["island","sex"], inplace=True)

    # drop rows with missing values
    mask = df.isna().sum(axis=1) == 0
    df = df[mask]
    tgt = tgt[mask]

    return df.values, tgt.to_numpy().reshape(-1,1)

def get_wine_dataset():
    # get data
    data = load_wine()
    X_full = data.data
    y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
    return X_full, y_full

def get_letters_dataset():
    # get data
    letter = fetch_openml('letter', version=1, as_frame=False)
    return letter.data, letter.target.reshape(-1,1)

def label_encode(y_full):
    return LabelEncoder().fit_transform(y_full.flatten()).reshape(y_full.shape)

def split_transpose(X, y, test_size, random_state):
    # X_train, X_test, y_train, y_test but all transposed
    return [elem.T for elem in train_test_split(X, y, test_size=test_size, random_state=random_state)]

## Benchmarking

Nota: esta clase fue creada bastante rápido y no pretende ser una plataforma súper confiable sobre la que basarse, sino más bien una herramienta simple con la que poder medir varios runs y agregar la información.

En forma rápida, `warmup` es la cantidad de runs para warmup, `mem_runs` es la cantidad de runs en las que se mide el pico de uso de RAM y `n_runs` es la cantidad de runs en las que se miden tiempos.

La razón por la que se separan es que medir memoria hace ~2.5x más lento cada run, pero al mismo tiempo se estabiliza mucho más rápido.

**Importante:** tener en cuenta que los modelos que predicen en batch (usan `predict` directamente) deberían consumir, como mínimo, $n$ veces la memoria de los que predicen por observación.

In [145]:
import time
from tqdm.notebook import tqdm
from numpy.random import RandomState
import tracemalloc

RNG_SEED = 6553

class Benchmark:
    def __init__(self, X, y, n_runs=1000, warmup=100, mem_runs=100, test_sz=0.3, rng_seed=RNG_SEED, same_splits=True):
        self.X = X
        self.y = y
        self.n = n_runs
        self.warmup = warmup
        self.mem_runs = mem_runs
        self.test_sz = test_sz
        self.det = same_splits
        if self.det:
            self.rng_seed = rng_seed
        else:
            self.rng = RandomState(rng_seed)

        self.data = dict()

        print("Benching params:")
        print("Total runs:",self.warmup+self.mem_runs+self.n)
        print("Warmup runs:",self.warmup)
        print("Peak Memory usage runs:", self.mem_runs)
        print("Running time runs:", self.n)
        approx_test_sz = int(self.y.size * self.test_sz)
        print("Train size rows (approx):",self.y.size - approx_test_sz)
        print("Test size rows (approx):",approx_test_sz)
        print("Test size fraction:",self.test_sz)

    def bench(self, model_class, **kwargs):
        name = model_class.__name__
        time_data = np.empty((self.n, 3), dtype=float)  # train_time, test_time, accuracy
        mem_data = np.empty((self.mem_runs, 2), dtype=float)  # train_peak_mem, test_peak_mem
        rng = RandomState(self.rng_seed) if self.det else self.rng


        for i in range(self.warmup):
            # Instantiate model with error check for unsupported parameters
            model = model_class(**kwargs)

            # Generate current train-test split
            X_train, X_test, y_train, y_test = split_transpose(
                self.X, self.y,
                test_size=self.test_sz,
                random_state=rng
            )
            # Run training and prediction (timing or memory measurement not recorded)
            model.fit(X_train, y_train)
            model.predict(X_test)

        for i in tqdm(range(self.mem_runs), total=self.mem_runs, desc=f"{name} (MEM)"):

            model = model_class(**kwargs)

            X_train, X_test, y_train, y_test = split_transpose(
                self.X, self.y,
                test_size=self.test_sz,
                random_state=rng
            )

            tracemalloc.start()

            t1 = time.perf_counter()
            model.fit(X_train, y_train)
            t2 = time.perf_counter()

            _, train_peak = tracemalloc.get_traced_memory()
            tracemalloc.reset_peak()

            model.predict(X_test)
            t3 = time.perf_counter()
            _, test_peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()

            mem_data[i,] = (
                train_peak / (1024 * 1024),
                test_peak / (1024 * 1024)
            )

        for i in tqdm(range(self.n), total=self.n, desc=f"{name} (TIME)"):
            model = model_class(**kwargs)

            X_train, X_test, y_train, y_test = split_transpose(
                self.X, self.y,
                test_size=self.test_sz,
                random_state=rng
            )

            t1 = time.perf_counter()
            model.fit(X_train, y_train)
            t2 = time.perf_counter()
            preds = model.predict(X_test)
            t3 = time.perf_counter()

            time_data[i,] = (
                (t2 - t1) * 1000,
                (t3 - t2) * 1000,
                (y_test.flatten() == preds.flatten()).mean()
            )

        self.data[name] = (time_data, mem_data)

    def summary(self, baseline=None):
        aux = []
        for name, (time_data, mem_data) in self.data.items():
            result = {
                'model': name,
                'train_mean_ms': time_data[:, 0].mean(),
                'train_std_ms': time_data[:, 0].std(),
                'test_mean_ms': time_data[:, 1].mean(),
                'test_std_ms': time_data[:, 1].std(),
                'mean_accuracy': time_data[:, 2].mean(),
                'train_mem_mean_mb': mem_data[:, 0].mean(),
                'train_mem_std_mb': mem_data[:, 0].std(),
                'test_mem_mean_mb': mem_data[:, 1].mean(),
                'test_mem_std_mb': mem_data[:, 1].std()
            }
            aux.append(result)
        df = pd.DataFrame(aux).set_index('model')

        if baseline is not None and baseline in self.data:
            df['train_speedup'] = df.loc[baseline, 'train_mean_ms'] / df['train_mean_ms']
            df['test_speedup'] = df.loc[baseline, 'test_mean_ms'] / df['test_mean_ms']
            df['train_mem_reduction'] = df.loc[baseline, 'train_mem_mean_mb'] / df['train_mem_mean_mb']
            df['test_mem_reduction'] = df.loc[baseline, 'test_mem_mean_mb'] / df['test_mem_mean_mb']
        return df

## Ejemplo

In [146]:
# levantamos el dataset Wine, que tiene 13 features y 178 observaciones en total
X_full, y_full = get_wine_dataset()

X_full.shape, y_full.shape

((178, 13), (178, 1))

In [147]:
# encodeamos a número las clases
y_full_encoded = label_encode(y_full)
print("y_full shape= ", y_full.shape)
print("y_full_encoded shape= ", y_full_encoded.shape)

y_full[:5], y_full_encoded[:5]

y_full shape=  (178, 1)
y_full_encoded shape=  (178, 1)


(array([['class_0'],
        ['class_0'],
        ['class_0'],
        ['class_0'],
        ['class_0']], dtype='<U7'),
 array([[0],
        [0],
        [0],
        [0],
        [0]], dtype=int64))

In [148]:
# generamos el benchmark
# observar que son valores muy bajos de runs para que corra rápido ahora
b = Benchmark(
    X_full, y_full_encoded,
    n_runs = 2,
    warmup = 2,
    mem_runs = 2,
    test_sz = 0.3,
    same_splits = False
)

Benching params:
Total runs: 6
Warmup runs: 2
Peak Memory usage runs: 2
Running time runs: 2
Train size rows (approx): 125
Test size rows (approx): 53
Test size fraction: 0.3


In [149]:
# bencheamos un par
to_bench = [QDA]

for model in to_bench:
    b.bench(model)

X shape=  (13, 124)
y shape=  (1, 124)
X shape=  (13, 124)
y shape=  (1, 124)


QDA (MEM):   0%|          | 0/2 [00:00<?, ?it/s]

X shape=  (13, 124)
y shape=  (1, 124)
X shape=  (13, 124)
y shape=  (1, 124)


QDA (TIME):   0%|          | 0/2 [00:00<?, ?it/s]

X shape=  (13, 124)
y shape=  (1, 124)
X shape=  (13, 124)
y shape=  (1, 124)


In [150]:
# como es una clase, podemos seguir bencheando más después
b.bench(TensorizedQDA)

X shape=  (13, 124)
y shape=  (1, 124)
X shape=  (13, 124)
y shape=  (1, 124)


TensorizedQDA (MEM):   0%|          | 0/2 [00:00<?, ?it/s]

X shape=  (13, 124)
y shape=  (1, 124)
X shape=  (13, 124)
y shape=  (1, 124)


TensorizedQDA (TIME):   0%|          | 0/2 [00:00<?, ?it/s]

X shape=  (13, 124)
y shape=  (1, 124)
X shape=  (13, 124)
y shape=  (1, 124)


In [151]:
b.bench(FasterQDA)

X shape=  (13, 124)
y shape=  (1, 124)
Shape de inner_prod: (3, 54, 54)
X shape=  (13, 124)
y shape=  (1, 124)
Shape de inner_prod: (3, 54, 54)


FasterQDA (MEM):   0%|          | 0/2 [00:00<?, ?it/s]

X shape=  (13, 124)
y shape=  (1, 124)
Shape de inner_prod: (3, 54, 54)
X shape=  (13, 124)
y shape=  (1, 124)
Shape de inner_prod: (3, 54, 54)


FasterQDA (TIME):   0%|          | 0/2 [00:00<?, ?it/s]

X shape=  (13, 124)
y shape=  (1, 124)
Shape de inner_prod: (3, 54, 54)
X shape=  (13, 124)
y shape=  (1, 124)
Shape de inner_prod: (3, 54, 54)


In [152]:
# hacemos un summary
b.summary()

,train_mean_ms,train_std_ms,test_mean_ms,test_std_ms,mean_accuracy,train_mem_mean_mb,train_mem_std_mb,test_mem_mean_mb,test_mem_std_mb
model,,,,,,,,,
QDA,1.62580,0.05520,2.37395,0.00255,0.981481,0.019442,0.000030,0.009154,0.000205
TensorizedQDA,2.02810,0.34410,2.01310,0.21870,0.953704,0.020076,0.001259,0.012485,0.000359
FasterQDA,1.76705,0.30005,0.14190,0.01010,0.981481,0.019454,0.000278,0.128331,0.000669


In [153]:
# son muchos datos! nos quedamos con un par nomás
summ = b.summary()

# como es un pandas DataFrame, subseteamos columnas fácil
summ[['train_mean_ms', 'test_mean_ms','mean_accuracy']]

,train_mean_ms,test_mean_ms,mean_accuracy
model,,,
QDA,1.62580,2.37395,0.981481
TensorizedQDA,2.02810,2.01310,0.953704
FasterQDA,1.76705,0.14190,0.981481


In [154]:
# podemos setear un baseline para que fabrique columnas de comparación
summ = b.summary(baseline='QDA')

summ

,train_mean_ms,train_std_ms,test_mean_ms,test_std_ms,mean_accuracy,train_mem_mean_mb,train_mem_std_mb,test_mem_mean_mb,test_mem_std_mb,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,,,,,,,
QDA,1.62580,0.05520,2.37395,0.00255,0.981481,0.019442,0.000030,0.009154,0.000205,1.000000,1.000000,1.000000,1.000000
TensorizedQDA,2.02810,0.34410,2.01310,0.21870,0.953704,0.020076,0.001259,0.012485,0.000359,0.801637,1.179251,0.968434,0.733252
FasterQDA,1.76705,0.30005,0.14190,0.01010,0.981481,0.019454,0.000278,0.128331,0.000669,0.920065,16.729739,0.999387,0.071334


In [155]:
summ[[
    'train_mean_ms', 'test_mean_ms','mean_accuracy',
    'train_speedup', 'test_speedup',
    'train_mem_reduction', 'test_mem_reduction'
]]

,train_mean_ms,test_mean_ms,mean_accuracy,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,
QDA,1.62580,2.37395,0.981481,1.000000,1.000000,1.000000,1.000000
TensorizedQDA,2.02810,2.01310,0.953704,0.801637,1.179251,0.968434,0.733252
FasterQDA,1.76705,0.14190,0.981481,0.920065,16.729739,0.999387,0.071334


# Consigna QDA

**Notación**: en general notamos

* $k$ la cantidad de clases
* $n$ la cantidad de observaciones
* $p$ la cantidad de features/variables/predictores

**Sugerencia:** combinaciones adecuadas de `transpose`, `stack`, `reshape` y, ocasionalmente, `flatten` y `diagonal` suele ser más que suficiente. Se recomienda **fuertemente* explorar la dimensionalidad de cada elemento antes de implementar las clases.

## Tensorización

En esta sección nos vamos a ocupar de hacer que el modelo sea más rápido para generar predicciones, observando que incurre en un doble `for` dado que predice en forma individual un escalar para cada observación, para cada clase. Paralelizar ambos vía tensorización suena como una gran vía de mejora de tiempos.

### 1) Diferencias entre `QDA`y `TensorizedQDA`

1. ¿Sobre qué paraleliza `TensorizedQDA`? ¿Sobre las $k$ clases, las $n$ observaciones a predecir, o ambas?
2. Analizar los shapes de `tensor_inv_covs` y `tensor_means` y explicar paso a paso cómo es que `TensorizedQDA` llega a predecir lo mismo que `QDA`.

### 2) Optimización

Debido a la forma cuadrática de QDA, no se puede predecir para $n$ observaciones en una sola pasada (utilizar $X \in \mathbb{R}^{p \times n}$ en vez de $x \in \mathbb{R}^p$) sin pasar por una matriz de $n \times n$ en donde se computan todas las interacciones entre observaciones. Se puede acceder al resultado recuperando sólo la diagonal de dicha matriz, pero resulta ineficiente en tiempo y (especialmente) en memoria. Aún así, es *posible* que el modelo funcione más rápido.

3. Implementar el modelo `FasterQDA` (se recomienda heredarlo de `TensorizedQDA`) de manera de eliminar el ciclo for en el método predict.
4. Mostrar dónde aparece la mencionada matriz de $n \times n$, donde $n$ es la cantidad de observaciones a predecir.
5. Demostrar que
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ es decir, que se puede "esquivar" la matriz de $n \times n$ usando matrices de $n \times p$. También se puede usar, de forma equivalente,
$$
np.sum(A^T \odot B, axis=0).T
$$
  queda a preferencia del alumno cuál usar.

6. Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente en un nuevo modelo `EfficientQDA`.
7. Comparar la performance de las 4 variantes de QDA implementadas hasta ahora (no Cholesky) ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

## Cholesky

Hasta ahora todos los esfuerzos fueron enfocados en realizar una predicción más rápida. Los tiempos de entrenamiento (teóricos al menos) siguen siendo los mismos o hasta (minúsculamente) peores, dado que todas las mejoras siguen llamando al método `_fit_params` original de `QDA`.

La descomposición/factorización de [Cholesky](https://en.wikipedia.org/wiki/Cholesky_decomposition#Statement) permite factorizar una matriz definida positiva $A = LL^T$ donde $L$ es una matriz triangular inferior. En particular, si bien se asume que $p \ll n$, invertir la matriz de covarianzas $\Sigma$ para cada clase impone un cuello de botella que podría alivianarse. Teniendo en cuenta que las matrices de covarianza son simétricas y salvo degeneración, definidas positivas, Cholesky como mínimo debería permitir invertir la matriz más rápido.

*Nota: observar que calcular* $A^{-1}b$ *equivale a resolver el sistema* $Ax=b$.

### 3) Diferencias entre implementaciones de `QDA_Chol`

8. Si una matriz $A$ tiene fact. de Cholesky $A=LL^T$, expresar $A^{-1}$ en términos de $L$. ¿Cómo podría esto ser útil en la forma cuadrática de QDA?
7. Explicar las diferencias entre `QDA_Chol1`y `QDA` y cómo `QDA_Chol1` llega, paso a paso, hasta las predicciones.
8. ¿Cuáles son las diferencias entre `QDA_Chol1`, `QDA_Chol2` y `QDA_Chol3`?
9. Comparar la performance de las 7 variantes de QDA implementadas hasta ahora ¿Qué se observa?¿Hay alguna de las implementaciones de `QDA_Chol` que sea claramente mejor que las demás?¿Alguna que sea peor?

### 4) Optimización

12. Implementar el modelo `TensorizedChol` paralelizando sobre clases/observaciones según corresponda. Se recomienda heredarlo de alguna de las implementaciones de `QDA_Chol`, aunque la elección de cuál de ellas queda a cargo del alumno según lo observado en los benchmarks de puntos anteriores.
13. Implementar el modelo `EfficientChol` combinando los insights de `EfficientQDA` y `TensorizedChol`. Si se desea, se puede implementar `FasterChol` como ayuda, pero no se contempla para el punto.
13. Comparar la performance de las 9 variantes de QDA implementadas ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

# Resolucion del TP

### 1) ¿Sobre qué paraleliza TensorizedQDA? ¿Sobre las $k$ clases, las $n$ observaciones a predecir, o ambas?

  TensorizedQDA paraleliza sobre ambas dimensiones: las $k$ clases y las $n$ observaciones a predecir.
  En el modelo QDA original, la predicción implica un doble bucle: uno sobre las $k$ clases y otro sobre las $n$ observaciones, calculando el score discriminante para cada observación y cada clase de forma individual. Esto resulta en una complejidad computacional que escala con $O(n \cdot k)$.
  En cambio, TensorizedQDA utiliza operaciones tensorizadas (con matrices y vectores) para eliminar estos bucles explícitos, aprovechando el paralelismo implícito de las operaciones matriciales en bibliotecas como NumPy. Esto permite calcular los scores para todas las observaciones y todas las clases simultáneamente en una sola pasada.

## Agustin:
Tensoriza sobre las clases. En la versión previa el predict_log_conditional actuaba para cada clase individualmente. De esta forma, elimina el for loop de predict_one que operaba sobre las clases (enumerando el vector a_priori)


### 2. Analizar los shapes de tensor_inv_covs y tensor_means y explicar paso a paso cómo es que TensorizedQDA llega a predecir lo mismo que QDA.

- Shapes esperados:
  - tensor_means: Shape $(k, p)$, donde $k$ es el número de clases y $p$ es el número de features. Representa los vectores de medias para cada clase.
  - tensor_inv_covs: Shape $(k, p, p)$, donde cada "capa" $(p, p)$ es la matriz inversa de la covarianza para una clase específica.

- Paso a paso en TensorizedQDA:
  1. Entrada: Se recibe un conjunto de observaciones $X$ de shape $(n, p)$, donde $n$ es el número de observaciones a predecir.
  2. Diferencia con las medias: Se calcula $X - \mu_k$ para cada clase $k$. Esto se hace restando tensor_means (expandido a $(n, k, p)$ mediante broadcasting) a $X$ (expandido a $(n, k, p)$), resultando en una matriz de diferencias de shape $(n, k, p)$.
  3. Forma cuadrática: Para cada clase $k$, se necesita calcular $(x_i - \mu_k)^T \Sigma_k^{-1} (x_i - \mu_k)$ para cada observación $x_i$. En TensorizedQDA, esto se logra usando tensor_inv_covs $(k, p, p)$ y las diferencias $(n, k, p)$ mediante una operación matricial tensorizada (como einsum), produciendo un tensor de shape $(n, k)$ con los términos cuadráticos para todas las observaciones y clases.
  4. Términos adicionales: Se suman los logaritmos de las probabilidades a priori (log_priors, shape $(k,)$) y los logaritmos de los determinantes de las covarianzas (calculados en el ajuste), ajustados para cada clase.
  5. Predicción: Se toma el argmax a lo largo del eje de las clases (dimensión $k$) para obtener las predicciones de shape $(n,)$, igual que en QDA.

- Equivalencia: La tensorización no cambia la fórmula matemática del discriminante cuadrático, solo reorganiza los cálculos para evitar bucles explícitos, logrando el mismo resultado que QDA.

## Agustin:

  - El shape de tensor_inv_covs es (cantidad de clases, cantidad de features, cantidad de features), siendo en este caso las clases 3 y las features 13, ya que para cada clase genera una matriz cuadrada de las covarianzas entre cada feature (siendo la diagonal la varianza propia de la feature). El shape de tensor_means es (cantidad de clases, cantidad de features, 1), ya que en este caso calcula una media para feature y para cada clase. El cambio con respecto a la versión previa es que antes cada matriz para cada clase se guardaba en una lista, y eran utilizadas dentro el for loop de predict_one, y ahora están agrupadas en un tensor. Para entender por qué llega al mismo resultado, debemos entender cómo funcionan las operaciones @ (np.matmul) y np.linalg.det(), cuando lo aplicamos sobre un tensor que consiste en k matrices apiladas. Lo que hacen ambas operaciones es aplicar la operación sobre las k matrices que están apiladas. Operación por operación ocurre lo siguiente:
  - Con unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_x, con shape de unbiased_x de (3,13,1) y tensor_inv_cov de (3,13,13): -unbiased_x.transpose(0,2,1) convierte el tensor a (3,1,13) -unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov multiplica el tensor de (3,1,13) con el de (3,13,13), donde internamente multiplica los 3 vectores de (1,13) por las 3 matrices de (13,13) y las vuelve a apilar. De esta forma, funciona igual que el for loop que lo hacía en forma separada para las 3, pero numpy lo hace en forma más eficiente al poder paralelizar la operación. El resultado es un tensor de (3,1,13) -Finalmente, el tensor de (3,1,13) es multiplicado por el tensor de unbiased_x resultando en un tensor de (3, 1, 1), siendo el segundo término de la log verosimilitud para cada una de las clases (al multiplicar por -0.5):

$\log f_j(x) = 12 \log|\Sigma_j| - 12(x-\mu_j)T\Sigma-1 j(x-\mu_j) + C$

 log𝑓𝑗(𝑥)=−12log|Σ𝑗|−12(𝑥−𝜇𝑗)𝑇Σ−1𝑗(𝑥−𝜇𝑗)+𝐶$ 
  - np.log(LA.det(self.tensor_inv_cov)) la función det opera de la misma forma, ya que tensor_inv_cov es de (3,13,13) por lo que para cada una de las 3 clases, genera el determinante de las matrices de (13,13), esto multiplicado por 0.5 nos da el primer término de la ecuación

  - Por último al sumar esto a la log_a_priori, tenemos las posterior, en formato tensor (3,1,1), y para predecir la clase se toma argmax La diferencia es que elimina el loop for de las clases, aplicando las operaciones mediante tensores



### 3. Implementar el modelo FasterQDA (se recomienda heredarlo de TensorizedQDA) de manera de eliminar el ciclo for en el método predict.

El siguiente código elimina el bucle explícito sobre observaciones al usar einsum para calcular la forma cuadrática en una sola operación tensorizada.

In [156]:
class FasterQDA(TensorizedQDA):
    def predict(self, X):
        m_obs = X.shape[1]
        k_classes = len(self.log_a_priori)
        
        # Calcular las log-posteriori para todas las observaciones y clases
        log_posteriori = np.empty((k_classes, m_obs))
        
        X_expandido = np.expand_dims(X, axis=0)  # (m_obs, 1, n_features, 1)
        X_expandido = np.repeat(X_expandido, repeats=len(self.log_a_priori), axis=0)
        
        # Diferencia entre X y la media de la clase k
        unbiased_X = X - self.tensor_means
        # Esto genera una tensor de (n_clases, m_obs, m_obs)
        inner_prod = unbiased_X.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_X
        print(f"Shape de inner_prod: {inner_prod.shape}")
        # Extraer solo la diagonal (log-probabilidad condicional para cada observación)
        log_conditional = 0.5 * np.log(LA.det(self.tensor_inv_cov)).reshape(-1, 1) - 0.5 * np.diagonal(inner_prod, axis1=1, axis2=2)
        # Sumar el log a priori
        log_posteriori = self.log_a_priori.reshape(-1, 1) + log_conditional
        
        # Elegir la clase con máxima probabilidad log-posteriori
        y_hat = np.argmax(log_posteriori, axis=0)
        return y_hat.reshape(1, -1)

In [157]:
class FasterQDA_v2(TensorizedQDA):
    #Implementación sin la matriz de nxn
    def _predict_log_conditionals(self,X):
        X = X.T # Pasar las observaciones a filas (m_obs, n_features)
        X = X.reshape(X.shape[0], X.shape[1],1) # Agregar 3ra dimension (m_obs, n_features, 1)
        X_expandido = np.expand_dims(X, axis=1)  # (m_obs, 1, n_features, 1)
        X_expandido = np.repeat(X_expandido, repeats=len(self.log_a_priori), axis=1)  # len(self.log_a_priori) es la cantidad de clases. Replica el vector para cada clase (m_obs, k_clases, n_features, 1)
        unbiased_X = X_expandido - self.tensor_means
        inner_prod = unbiased_X.transpose(0,1,3,2) @ self.tensor_inv_cov @ unbiased_X
        return 0.5*np.log(LA.det(self.tensor_inv_cov)) - 0.5 * inner_prod.squeeze()

    def _predict_one(self, X):
        # return the class that has maximum a posteriori probability
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(X), axis =1)
    
    def predict(self, X):
        # this is actually an individual prediction encased in a for-loop
        y_hat = self._predict_one(X)

        # return prediction as a row vector (matching y)
        return y_hat.reshape(1,-1)

In [158]:
b.bench(FasterQDA)

X shape=  (13, 124)
y shape=  (1, 124)
Shape de inner_prod: (3, 54, 54)
X shape=  (13, 124)
y shape=  (1, 124)
Shape de inner_prod: (3, 54, 54)


FasterQDA (MEM):   0%|          | 0/2 [00:00<?, ?it/s]

X shape=  (13, 124)
y shape=  (1, 124)
Shape de inner_prod: (3, 54, 54)
X shape=  (13, 124)
y shape=  (1, 124)
Shape de inner_prod: (3, 54, 54)


FasterQDA (TIME):   0%|          | 0/2 [00:00<?, ?it/s]

X shape=  (13, 124)
y shape=  (1, 124)
Shape de inner_prod: (3, 54, 54)
X shape=  (13, 124)
y shape=  (1, 124)
Shape de inner_prod: (3, 54, 54)


In [159]:
b.bench(FasterQDA_v2)

X shape=  (13, 124)
y shape=  (1, 124)
X shape=  (13, 124)
y shape=  (1, 124)


FasterQDA_v2 (MEM):   0%|          | 0/2 [00:00<?, ?it/s]

X shape=  (13, 124)
y shape=  (1, 124)
X shape=  (13, 124)
y shape=  (1, 124)


FasterQDA_v2 (TIME):   0%|          | 0/2 [00:00<?, ?it/s]

X shape=  (13, 124)
y shape=  (1, 124)
X shape=  (13, 124)
y shape=  (1, 124)


In [160]:
b.summary()

,train_mean_ms,train_std_ms,test_mean_ms,test_std_ms,mean_accuracy,train_mem_mean_mb,train_mem_std_mb,test_mem_mean_mb,test_mem_std_mb
model,,,,,,,,,
QDA,1.62580,0.05520,2.37395,0.00255,0.981481,0.019442,0.000030,0.009154,0.000205
TensorizedQDA,2.02810,0.34410,2.01310,0.21870,0.953704,0.020076,0.001259,0.012485,0.000359
FasterQDA,1.89215,0.03955,0.26190,0.00230,0.990741,0.018661,0.000675,0.128058,0.000550
FasterQDA_v2,2.20025,0.62505,0.18180,0.04970,0.981481,0.018432,0.000507,0.060978,0.000520


### 4) Mostrar dónde aparece la mencionada matriz de $n \times n$, donde $n$ es la cantidad de observaciones a predecir.



  - La matriz de $n \times n$ aparece en el término cuadrático si intentamos calcularlo directamente como $(X - M)^T \Sigma^{-1} (X - M)$, donde $X$ es $(n, p)$ y $M$ es una matriz de medias repetidas para cada observación. En FasterQDA, esto ocurre en el tensor denominado inner_product (el tensor queda de la forma (k, n, n))
    - diffs $(n, k, p)$ y tensor_inv_covs $(k, p, p)$ generan un producto intermedio que, si no se reduce inmediatamente, podría interpretarse como una interacción entre todas las $n$ observaciones para cada clase, pero np.diag colapsa esto a $(k, n)$ directamente, evitando materializar la matriz $n \times n$ completa.

### 5) **Demostrar que**  $\text{diag}(A \cdot B) = \sum_{\text{cols}} A \odot B^T = \texttt{np.sum}(A \odot B^T, \texttt{axis=1})$

**Prueba:**

Sea $ A \in \mathbb{R}^{n \times k}$  y $B \in \mathbb{R}^{k \times n}$.

El producto $( A \cdot B ) \in \mathbb{R}^{n \times n} $, por lo que su diagonal:

$$
\text{diag}(A \cdot B)_i = \sum_{j=1}^{k} A_{ij} B_{ji}
$$

Por definición de multiplicación de matrices:

$$
(A \cdot B)_{ii} = \sum_{j=1}^{k} A_{ij} \cdot B_{ji}
$$

Por lo tanto:

$$
\text{diag}(A \cdot B)_i = \sum_{j=1}^{k} A_{ij} \cdot B_{ji}
$$

Dado que $( B^T \in \mathbb{R}^{n \times k} )$, entonces $( A \odot B^T \in \mathbb{R}^{n \times k} )$

El elemento en la posición \( (i, j) \) de \( A \odot B^T \) es:

$$
(A \odot B^T)_{ij} = A_{ij} \cdot B^T_{ij} = A_{ij} \cdot B_{ji}
$$

Entonces:

$$
\sum_{\text{cols}} (A \odot B^T)_i = \sum_{j=1}^{k} A_{ij} \cdot B_{ji}
$$

Lo que equivale a la expresión para \( \text{diag}(A \cdot B) \)

Por lo tanto:

$$
\text{diag}(A \cdot B) = \texttt{np.sum}(A \odot B^T, \texttt{axis=1})
$$

En NumPy, dada una matriz `A` de dimensiones `(n, k)` y `B` con dimensiones `(k, n)`,  
se puede calcular la diagonal de `A @ B` por medio de:

```python
np.sum(A * B.T, axis=1)

Debido a que 
  - A * B.T es la multiplicación elemento a elemento
  - np.sum(..., axis=1) suma en forma columnar

### 6) Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo FasterQDA de forma eficiente en un nuevo modelo EfficientQDA.

In [161]:
class EfficientQDA(TensorizedQDA):
    def predict(self, X):
        m_obs = X.shape[1]
        k_classes = len(self.log_a_priori)
        
        # Calcular las log-posteriori para todas las observaciones y clases
        log_posteriori = np.empty((k_classes, m_obs))
        
        X_expandido = np.expand_dims(X, axis=0)  # (m_obs, 1, n_features, 1)
        X_expandido = np.repeat(X_expandido, repeats=len(self.log_a_priori), axis=0)
        
        # Diferencia entre X y la media de la clase k
        unbiased_X = X - self.tensor_means
        # Calcular inv_cov_k @ (X - mu_k)
        cov_unbiased = self.tensor_inv_cov @ unbiased_X
        # Usar la propiedad: suma de (X - mu_k)^T ⊙ (inv_cov_k @ (X - mu_k))
        inner_prod = np.sum(unbiased_X.T * cov_unbiased.T, axis=1)
        # Extraer solo la diagonal (log-probabilidad condicional para cada observación)
        log_conditional = 0.5 * np.log(LA.det(self.tensor_inv_cov)).reshape(-1, 1) - 0.5 * inner_prod.T
        # Sumar el log a priori
        log_posteriori = self.log_a_priori.reshape(-1, 1) + log_conditional
        
        # Elegir la clase con máxima probabilidad log-posteriori
        y_hat = np.argmax(log_posteriori, axis=0)
        return y_hat.reshape(1, -1)

In [162]:
b.bench(EfficientQDA)

X shape=  (13, 124)
y shape=  (1, 124)
X shape=  (13, 124)
y shape=  (1, 124)


EfficientQDA (MEM):   0%|          | 0/2 [00:00<?, ?it/s]

X shape=  (13, 124)
y shape=  (1, 124)
X shape=  (13, 124)
y shape=  (1, 124)


EfficientQDA (TIME):   0%|          | 0/2 [00:00<?, ?it/s]

X shape=  (13, 124)
y shape=  (1, 124)
X shape=  (13, 124)
y shape=  (1, 124)


In [163]:
b.summary()

,train_mean_ms,train_std_ms,test_mean_ms,test_std_ms,mean_accuracy,train_mem_mean_mb,train_mem_std_mb,test_mem_mean_mb,test_mem_std_mb
model,,,,,,,,,
QDA,1.62580,0.05520,2.37395,0.00255,0.981481,0.019442,0.000030,0.009154,0.000205
TensorizedQDA,2.02810,0.34410,2.01310,0.21870,0.953704,0.020076,0.001259,0.012485,0.000359
FasterQDA,1.89215,0.03955,0.26190,0.00230,0.990741,0.018661,0.000675,0.128058,0.000550
FasterQDA_v2,2.20025,0.62505,0.18180,0.04970,0.981481,0.018432,0.000507,0.060978,0.000520
EfficientQDA,1.64790,0.22580,0.34280,0.25520,0.981481,0.019671,0.000348,0.094943,0.000194


### 7) Comparar la performance de las 4 variantes de QDA implementadas hasta ahora (no Cholesky) ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

| Aspecto                  | QDA                              | TensorizedQDA                    | FasterQDA_v2                     | EfficientQDA                     |
|--------------------------|----------------------------------|----------------------------------|----------------------------------|----------------------------------|
| **Bucle sobre observaciones** | Sí $(n)$                    | Sí $(n)$                       | No                               | No                               |
| **Bucle sobre clases**   | Sí (\(k\))                      | No                              | No                               | No                               |
| **Uso de tensorización** | No                              | Sí (sobre clases)               | Sí (sobre clases y observaciones) | Sí (sobre clases y observaciones) |
| **Complejidad**          | $O(n \cdot k \cdot p^2)$      | $O(n \cdot p^2)$              | $O(p^2 \cdot k)$               | $O(p^2 \cdot k)$               |
| **Predicción en lote**   | No                              | No                              | Sí                               | Sí                               |
| **Matriz $(n \times n)$** | No evitada                     | No evitada                     | Evitada                          | Evitada                          |
| **Cálculo de log-determinantes** | Correcto (por clase)         | Correcto (por clase)            | Incorrecto (tensor 3D); necesita corrección | Incorrecto (tensor 3D); necesita corrección |
| **Estrategia de optimización** | Bucles explícitos             | Tensorización parcial (clases)  | Tensorización completa, términos cuadráticos con `@` | Tensorización completa, suma de productos para términos cuadráticos |

**Notas:**
- \(n\): número de observaciones, \(k\): número de clases, \(p\): número de características.
- **QDA**: Implementación base con bucles sobre observaciones y clases, menos eficiente para grandes \(n\) y \(k\).
- **TensorizedQDA**: Elimina el bucle sobre clases usando tensorización, pero aún itera sobre observaciones.
- **FasterQDA_v2**: Procesa todas las observaciones y clases simultáneamente con operaciones matriciales batcheadas, pero requiere corregir el cálculo de `log(LA.det(self.tensor_inv_cov))` almacenando `self.log_dets` en `_fit_params`.
- **EfficientQDA**: Similar a `FasterQDA_v2`, usa tensorización completa y optimiza términos cuadráticos con suma de productos, pero también necesita corregir el cálculo de log-determinantes.
- Para ambos, `FasterQDA_v2` y `EfficientQDA`, se recomienda implementar:
  ```python
  def _fit_params(self, X, y):
      super()._fit_params(X, y)
      self.log_dets = np.array([np.log(np.linalg.det(self.covs_[k])) for k in range(len(self.log_a_priori))])

## Cholesky


### 3) Diferencias entre implementaciones de `QDA_Chol`

8. Si una matriz $A$ tiene fact. de Cholesky $A=LL^T$, expresar $A^{-1}$ en términos de $L$. ¿Cómo podría esto ser útil en la forma cuadrática de QDA?

La matriz $A^{-1}=(LL^T)^{-1}=L^{-T}L^{-1}$

Para aplicar Cholesky, requiero que la matriz sea simétrica y definida positiva. La primera condición se cumple siempre para la matriz de covarianza, mientras que la segunda siempre que no haya dependencia lineal exacta entre variables. Suponiendo que las condiciones se cumplen, puedo expresar:

$\Sigma_j^{-1} = L_j^{-T}L_j^{-1}$

Recordando que teníamos:
$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$

*   El primer término es:
    $$ -\frac{1}{2}\log |\Sigma_j|= -\frac{1}{2}\log |L_jL_j^T|=\frac{1}{2}\log (|L_j||L_j^T|)=-\frac{1}{2}\log (|L_j||L_j|)=\frac{1}{2}\log (|L_j|^2)=-\log (|L_j|)$$
    
    Y el determinante de una matriz diagonal es el producto de los valores de la diagonal:
    $$|L_j|=\prod_{i=1}^{n} L_{ii}$$

    En la aplicación en código, se calcula el determinante sobre la inversa, por lo que cancela el signo negativo

*   Reemplazando el segundo término de $\log{f_j(x)}$ con esta definición:
    $$\frac{1}{2}(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)=\frac{1}{2}(x-\mu_j)^T  L_j^{-T}L_j^{-1} (x- \mu_j)$$

    Y si defino $y=L^{-1} (x- \mu_j)$, obtengo que:
    $$\frac{1}{2}(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)=\frac{1}{2}(x-\mu_j)^T  L_j^{-T}L_j^{-1} (x- \mu_j)=\frac{1}{2}y^Ty=\frac{1}{2}\|y\|^2$$

Esto significa que puedo obtener el ambos términos de $\log{f_j(x)}$ calculando únicamente la matriz $L_j^{-1}$, sin necesidad de reconstruir la inversa completa $\Sigma_j^{-1}$. Y como $L_j$ es una matriz triangular, invertirla es más barato computacionalmente